In [29]:
import re
import math
from collections import Counter
import pymorphy3

class TerminologyExtractor:
    def __init__(self):
        self.morph = pymorphy3.MorphAnalyzer()
        self.stop_words = {'другой', 'этот', 'такой', 'какой-то', 'какой', 'таков', 'некий','весь','один','любой','разный','каждый','общий','некоторый','первый','мой', 'ваш','наш','второй','иной'}
        
    def load_text(self, filename):
        with open(filename, 'r', encoding='utf-8') as file:
            return file.read()
    
    def split_sentences(self, text):
        sentences = re.split(r'[.!?]+', text)
        return [sentence.strip() for sentence in sentences if sentence.strip()]
    
    def tokenize_words(self, text):
        words = re.findall(r'[а-яё-]+', text.lower())
        return words
    
    def extract_adjective_noun_pairs(self, text):
        sentences = self.split_sentences(text)
        pairs = []
        
        for sentence in sentences:
            words = self.tokenize_words(sentence)
            
            for i in range(len(words) - 1):
                try:
                    current_word = self.morph.parse(words[i])[0]
                    next_word = self.morph.parse(words[i+1])[0]
                    
                    if ('ADJF' in current_word.tag or 'ADJS' in current_word.tag) and 'NOUN' in next_word.tag:
        
                        if (current_word.normal_form not in self.stop_words and 
                            next_word.normal_form not in self.stop_words):
                            pair = (current_word.normal_form, next_word.normal_form)
                            pairs.append(pair)
                except:
                    continue
        
        return pairs
    
    def calculate_frequency_ranking(self, pairs):
        counter = Counter(pairs)
        return counter.most_common()
    
    def calculate_mi_ranking(self, pairs):
        pair_freq = Counter(pairs)
        adj_freq = Counter()
        noun_freq = Counter()
        
        for adj, noun in pairs:
            adj_freq[adj] += 1
            noun_freq[noun] += 1
        
        total_pairs = len(pairs)
        mi_scores = {}
        
        for (adj, noun), pair_count in pair_freq.items():
            if pair_count < 2:
                continue
                
            p_pair = pair_count / total_pairs
            p_adj = adj_freq[adj] / total_pairs
            p_noun = noun_freq[noun] / total_pairs
            
            if p_adj * p_noun > 0:
                mi = math.log2(p_pair / (p_adj * p_noun))
                mi_scores[(adj, noun)] = mi
        
        return sorted(mi_scores.items(), key=lambda x: x[1], reverse=True)
    
    def calculate_mi3_ranking(self, pairs):
        pair_freq = Counter(pairs)
        adj_freq = Counter()
        noun_freq = Counter()
        
        for adj, noun in pairs:
            adj_freq[adj] += 1
            noun_freq[noun] += 1
        
        total_pairs = len(pairs)
        mi3_scores = {}
        
        for (adj, noun), pair_count in pair_freq.items():
            if pair_count < 2: 
                continue
                
            p_pair = pair_count**3 / total_pairs
            p_adj = adj_freq[adj] / total_pairs
            p_noun = noun_freq[noun] / total_pairs
            
            if p_adj * p_noun > 0:
                mi = math.log2(p_pair / (p_adj * p_noun))
                mi3 = mi
                mi3_scores[(adj, noun)] = mi3
        
        return sorted(mi3_scores.items(), key=lambda x: x[1], reverse=True)
    
    def is_terminology(self, pair, domain_keywords):
        adj, noun = pair
        
        common_adjectives = {
            'большой', 'маленький', 'хороший', 'плохой', 'новый', 'старый',
            'красивый', 'интересный', 'важный', 'основной', 'разный', 'первый',
            'последний', 'каждый', 'некоторый', 'любой', 'весь', 'другой'
        }
        
        common_nouns = {
            'время', 'человек', 'дело', 'жизнь', 'день', 'рука', 'работа',
            'слово', 'место', 'лицо', 'друг', 'глаз', 'вопрос', 'дом', 'вид',
            'голова', 'сторона', 'мир', 'случай', 'ребенок', 'сила', 'конец'
        }
    
        
        if adj in common_adjectives:
            return False
            
        if noun in common_nouns:
            return False
            
        domain_indicators = any(
            keyword in adj or keyword in noun 
            for keyword in domain_keywords
        )
        
        return domain_indicators
    
    def analyze_results(self, frequency_ranking, mi_ranking, mi3_ranking, domain_keywords, top_n=20):
        print("=" * 80)
        print("РЕЗУЛЬТАТЫ ИЗВЛЕЧЕНИЯ ТЕРМИНОВ")
        print("=" * 80)
    
        # Для частотности
        print(f"\nТОП-{top_n} ПО ЧАСТОТНОСТИ:")
        print("-" * 50)
        freq_terms = []
        freq_non_terms = []
    
        for i, ((adj, noun), freq) in enumerate(frequency_ranking[:top_n], 1):
            is_term = self.is_terminology((adj, noun), domain_keywords)
            if is_term:
                freq_terms.append((adj, noun, freq))
                print(f"{i:2d}. ✓ {adj} {noun} (freq: {freq}) - ТЕРМИН")
            else:
                freq_non_terms.append((adj, noun, freq))
                print(f"{i:2d}. ✗ {adj} {noun} (freq: {freq}) - не термин")
    
        # Для MI
        print(f"\nТОП-{top_n} ПО MI:")
        print("-" * 50)
        mi_terms = []
        mi_non_terms = []
    
        for i, ((adj, noun), mi) in enumerate(mi_ranking[:top_n], 1):
            is_term = self.is_terminology((adj, noun), domain_keywords)
            if is_term:
                mi_terms.append((adj, noun, mi))
                print(f"{i:2d}. ✓ {adj} {noun} (MI: {mi:.3f}) - ТЕРМИН")
            else:
                mi_non_terms.append((adj, noun, mi))
                print(f"{i:2d}. ✗ {adj} {noun} (MI: {mi:.3f}) - не термин")
    
        # Для MI³
        print(f"\nТОП-{top_n} ПО MI³:")
        print("-" * 50)
        mi3_terms = []
        mi3_non_terms = []
    
        for i, ((adj, noun), mi3) in enumerate(mi3_ranking[:top_n], 1):
            is_term = self.is_terminology((adj, noun), domain_keywords)
            if is_term:
                mi3_terms.append((adj, noun, mi3))
                print(f"{i:2d}. ✓ {adj} {noun} (MI³: {mi3:.3f}) - ТЕРМИН")
            else:
                mi3_non_terms.append((adj, noun, mi3))
                print(f"{i:2d}. ✗ {adj} {noun} (MI³: {mi3:.3f}) - не термин")
    
        print("\n" + "=" * 80)
        print("СРАВНИТЕЛЬНЫЙ АНАЛИЗ:")
        print(f"Метод частотности: {len(freq_terms)} терминов из {top_n} ({len(freq_terms)/top_n*100:.1f}%)")
        print(f"Метод MI: {len(mi_terms)} терминов из {top_n} ({len(mi_terms)/top_n*100:.1f}%)")
        print(f"Метод MI³: {len(mi3_terms)} терминов из {top_n} ({len(mi3_terms)/top_n*100:.1f}%)")
    
        # Определяем победителя
        counts = {
            'ЧАСТОТНОСТИ': len(freq_terms),
            'MI': len(mi_terms),
            'MI³': len(mi3_terms)
        }
        winner = max(counts, key=counts.get)
    
        # Проверяем, нет ли ничьей
        max_count = counts[winner]
        winners = [method for method, count in counts.items() if count == max_count]
        if len(winners) > 1:
            winner = "НИЧЬЯ МЕЖДУ " + " И ".join(winners)
        else:
            winner = winner
    
        print(f"\nПобедитель: метод {winner}")
    
        return len(freq_terms), len(mi_terms), len(mi3_terms), freq_terms, mi_terms, mi3_terms

In [30]:
def main():
    extractor = TerminologyExtractor()
    
    text = extractor.load_text('book.txt')
    
    domain_keywords = {
    'сет', 'сетев', 'компьютер', 'информац', 'данн', 'протокол', 'транспорт', 
    'передач', 'соединен', 'коммутац', 'маршрутизац', 'адрес', 'пакет', 'трафик',
    'локальн', 'глобальн', 'корпоративн', 'виртуальн', 'беспроводн', 'проводн',
    'одноранг', 'клиент-сервер', 'распределен', 'централизован',
    'тсп', 'ip', 'udp', 'dns', 'dhcp', 'http', 'https', 'ftp', 'smtp', 'pop3',
    'imap', 'ssh', 'ssl', 'tls', 'voip', 'icmp', 'arp', 'rarp', 'nat', 'pat',
    'маршрутизатор', 'коммутатор', 'концентратор', 'мост', 'шлюз', 'модем',
    'точка-доступа', 'файрвол', 'межсетевой', 'трансивер', 'повторитель',
    'osi', 'tcp-ip', 'эталонн', 'взаимодейств', 'уровнев', 'прикладн',
    'представительск', 'сеансов', 'транспортн', 'сетев', 'канальн', 'физическ',
    'пропускн', 'скорост', 'задержк', 'jitter', 'надежн', 'доступн',
    'масштабируем', 'безопасн', 'отказоустойчив', 'качеств',
    'кабель', 'оптическ', 'витая', 'коаксиальн', 'волокон', 'радио',
    'инфракрасн', 'спутников', 'мобильн', 'широкополосн', 'узкополосн',
    'ip-адрес', 'mac-адрес', 'домен', 'dns-сервер', 'dhcp-сервер', 'подсет',
    'маска', 'шлюз-по-умолчанию', 'порт', 'сокет',
    'защит', 'криптограф', 'шифрован', 'аутентификац', 'авторизац', 'брандмауэр',
    'программ', 'вычисл', 'памят', 'алгоритм', 'квант', 'искусствен', 'интеллект',
    'техник', 'цифр', 'логич', 'баз', 'систем', 'технолог',
    'криптограф', 'нейрон', 'виртуал', 'мобиль', 'облач', 'распредел', 'параллель', 'реляцион', 'диск', 'ключ','меню','диапазон'
    }
    
    print("Извлечение конструкций 'прилагательное-существительное'...")
    pairs = extractor.extract_adjective_noun_pairs(text)
    
    print(f"Извлечено {len(pairs)} конструкций")
    print("\nПримеры извлеченных конструкций:")
    for i, (adj, noun) in enumerate(pairs[:15], 1):
        print(f"{i}. {adj} {noun}")
    
    print("\n" + "="*80)
    print("УПОРЯДОЧИВАНИЕ РЕЗУЛЬТАТОВ")
    print("="*80)
    
    frequency_ranking = extractor.calculate_frequency_ranking(pairs)
    mi_ranking = extractor.calculate_mi_ranking(pairs)
    mi3_ranking = extractor.calculate_mi3_ranking(pairs)
    
    freq_count, mi_count, mi3_count, freq_terms, mi_terms, mi3_terms = extractor.analyze_results(
        frequency_ranking, mi_ranking, mi3_ranking, domain_keywords
    )
    
    print("\n" + "="*80)
    print("СТАТИСТИКА:")
    print(f"Всего извлечено конструкций: {len(pairs)}")
    print(f"Уникальных сочетаний: {len(set(pairs))}")
    print(f"Эффективность метода частотности: {freq_count}/20 ({freq_count/20*100:.1f}%)")
    print(f"Эффективность метода MI: {mi_count}/20 ({mi_count/20*100:.1f}%)")
    print(f"Эффективность метода MI³: {mi3_count}/20 ({mi3_count/20*100:.1f}%)")

if __name__ == "__main__":
    main()

Извлечение конструкций 'прилагательное-существительное'...
Извлечено 411 конструкций

Примеры извлеченных конструкций:
1. электронный библиотека
2. приятный чтение
3. компьютерный сеть
4. ключевой элемент
5. уникальный имя
6. компьютерный сеть
7. локальный сеть
8. эталонный модель
9. коаксиальный кабель
10. полудуплексный режим
11. полнодуплексный сеть
12. витой пара
13. основный адрес
14. основный задача
15. однозначный идентификация

УПОРЯДОЧИВАНИЕ РЕЗУЛЬТАТОВ
РЕЗУЛЬТАТЫ ИЗВЛЕЧЕНИЯ ТЕРМИНОВ

ТОП-20 ПО ЧАСТОТНОСТИ:
--------------------------------------------------
 1. ✓ локальный сеть (freq: 25) - ТЕРМИН
 2. ✓ беспроводный сеть (freq: 17) - ТЕРМИН
 3. ✓ компьютерный сеть (freq: 14) - ТЕРМИН
 4. ✓ сетевой подключение (freq: 14) - ТЕРМИН
 5. ✗ правый кнопка (freq: 11) - не термин
 6. ✗ рабочий группа (freq: 7) - не термин
 7. ✓ исходящий трафик (freq: 7) - ТЕРМИН
 8. ✗ хостовый часть (freq: 6) - не термин
 9. ✓ проводной сеть (freq: 6) - ТЕРМИН
10. ✓ домашний сеть (freq: 6) - ТЕРМИН
11